## In-Vantage Scoring via PMML

As part of the PIMA Diabetes model, we produce a PMML representation of the xgboost model with the following code

```python
from nyoka import xgboost_to_pmml
...
xgboost_to_pmml(pipeline=model, col_names=feature_names, target_name=target_name, pmml_f_name="artifacts/output/model.pmml")
```

You can deploy that In-Vantage via the AnalyticOps UI, just as with RESTFul and Batch deployments. This notebook shows you how you can then use this to score models via SQL. 

In [1]:
import os
import pandas as pd
import getpass

from teradataml import create_context, remove_context
from teradataml.context.context import get_connection

In [2]:
password = getpass.getpass("Password")

engine = create_context(host="40.117.178.55", username="aoa_service", password=password, database="aoa_data")
conn = get_connection()
cursor = engine.raw_connection().cursor()

Password ···········


## View Published Models

We can view the models published to vantage by querying the table they are published to. Note this information is available via the AOA APIs also

In [3]:
pd.options.display.max_colwidth = 250
pd.read_sql("select * from aoa_ivsm_models", conn)

,model_version,model_id,model_type,project_id,deployed_at,model
0,dbc5f29e-ac76-441e-8025-9344307f635f,f937b5d8-02c6-5150-80c7-1e4ff07fea31,PMML,23e1df4b-b630-47a1-ab80-7ad5385fcd8d,2022-01-10 19:08:24.130,"b'<?xml version=""1.0"" encoding=""UTF-8""?>\n<PMML xmlns=""http://www.dmg.org/PMML-4_4"" version=""4.4"">\n <Header copyright=""Copyright (c) 2018 Software AG"" description=""Default Description"">\n <Application name=""Nyoka"" version=""4.3.0""/>\n ..."
1,df300e1a-781b-4b3e-9b8e-f32ef54c7a89,73d3f5a1-a5a9-55fd-9fe8-298a8aa7929d,PMML,a85ede91-9b28-4a7d-bb32-eee2aaffa82f,2022-01-10 19:09:58.490,"b'<?xml version=""1.0"" encoding=""UTF-8""?>\n<PMML xmlns=""http://www.dmg.org/PMML-4_4"" version=""4.4"">\n <Header copyright=""Copyright (c) 2018 Software AG"" description=""Default Description"">\n <Application name=""Nyoka"" version=""4.3.0""/>\n ..."


## On-Demand Scoring

In [5]:
query="""
SELECT * FROM IVSM.IVSM_SCORE(
    on PIMA_TEST
    on (SELECT model_id, model FROM aoa_ivsm_models WHERE model_version = 'dbc5f29e-ac76-441e-8025-9344307f635f') AS ModelTable DIMENSION
    using
        ModelID('f937b5d8-02c6-5150-80c7-1e4ff07fea31')
        ColumnsToPreserve('PatientId')
        ModelType('PMML')
) sc;
"""
pd.read_sql(query, conn).head()

,PatientId,score_result
0,3,"{""probability_0"":0.9985361840112014,""probability_1"":0.0014638159887985821,""predicted_HasDiabetes"":0,""probability"":0.9985361840112014}"
1,4,"{""probability_0"":0.31451604819478785,""probability_1"":0.6854839518052122,""predicted_HasDiabetes"":1,""probability"":0.6854839518052122}"
2,5,"{""probability_0"":0.9527500266004268,""probability_1"":0.04724997339957319,""predicted_HasDiabetes"":0,""probability"":0.9527500266004268}"
3,21,"{""probability_0"":0.8676194791001057,""probability_1"":0.13238052089989427,""predicted_HasDiabetes"":0,""probability"":0.8676194791001057}"
4,23,"{""probability_0"":0.13552471706313474,""probability_1"":0.8644752829368653,""predicted_HasDiabetes"":1,""probability"":0.8644752829368653}"


Copyright 2021 Teradata. All rights reserved.